<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_python_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python**

In [11]:
%%writefile tp2.py
import sys
import os
import random
from threading import Thread
MIN_MATRIX_SIZE = 5
MAX_MATRIX_SIZE = 20
MIN_MATRIX_CELL_VALUE = -32
MAX_MATRIX_CELL_VALUE = 32

def validateArgs(args):
    if not len(args) == 2:
        print("Cantidad incorrecta de argumentos")
        os._exit(1)
    size = int(args[1])
    if size > MAX_MATRIX_SIZE or size < MIN_MATRIX_SIZE:
        print("El tamaño debe estar entre {min} y {max}".format(
          min=MIN_MATRIX_SIZE,
          max=MAX_MATRIX_SIZE))
        os._exit(1)
    return size

def matrixGenerator(size):
    matrixResult=[]
    for i in range(0, size):
        matrixRow=[]
        for j in range(0, size):
            matrixRow.append(random.randint(
              MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE))
        matrixResult.append(matrixRow)
    return matrixResult

def printMatrix(alias,matrix):
    print("\nMatrix: {alias}".format(alias=alias))
    for row in matrix:
        for col in row:
            print("\t{col}\t".format(col=col),end="")
        print("\n",end="")

def secuentialMult(size, matrix_a, matrix_b):
    matrixResult = []
    for i in range(0, size):
        rowResult = [None] * size
        for j in range(0,size):
            for k in range(0,size):
                rowResult[j] = matrix_a[i][k] * matrix_b[k][j]
        matrixResult.append(rowResult)
    return matrixResult

## Funcion que ejecuta el thread
def multiplyRow(size, matrix_a, matrix_b, rowResult, rowIndex):
    for j in range(0,size):
        for k in range(0,size):
            rowResult[j]= matrix_a[rowIndex][k]*matrix_b[k][j]

## Genera un thread por cada fila encargado de ejecutar la multiplicacion y generar la fila resultante
def concurrentMult(size, matrix_a, matrix_b):
    matrixResult = []
    threads = []

    for i in range(0, size):
        rowResult = [None] * size
        thread = Thread(
          target=multiplyRow,
          args=(size, matrix_a, matrix_b, rowResult, i))
        thread.start()
        threads.append(thread)
        matrixResult.append(rowResult)

    for thread in threads:
        thread.join()

    return matrixResult

def compareMatrix(size, matrix_a, matrix_b):
    for i in range(0, size):
        for j in range(0, size):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True

def main():
    size = validateArgs(sys.argv)
    matrix_a = matrixGenerator(size)
    matrix_b = matrixGenerator(size)
    matrix_secuential = secuentialMult(size,matrix_a,matrix_b)
    matrix_concurrent = concurrentMult(size,matrix_a,matrix_b)

    printMatrix("A", matrix_a)
    printMatrix("B", matrix_b)
    printMatrix("Secuencial", matrix_secuential)
    printMatrix("Concurrente", matrix_concurrent)

    ## Determinamos si las matrices son iguales
    resultCompare = (
      "iguales" 
      if compareMatrix(size, matrix_secuential, matrix_concurrent)
      else "distintas")
    print("Las matrices concurrente y secuencial son {compare}".format(
      compare=resultCompare))

main()

Overwriting tp2.py


In [12]:
!python3 tp2.py 1

El tamaño debe estar entre 5 y 20


In [13]:
!python3 tp2.py 7


Matrix: A
	9		21		-17		-22		-4		30		2	
	3		31		-29		7		-21		23		29	
	18		-1		-6		17		29		-15		-2	
	13		-3		-27		-19		-8		0		7	
	-7		-22		18		16		-4		-8		-2	
	12		18		15		28		-21		-30		27	
	2		-29		-6		-5		-19		-4		2	

Matrix: B
	28		11		3		-4		-13		22		12	
	17		21		-31		-31		-7		28		-26	
	-21		14		27		25		19		7		-4	
	15		29		27		10		-5		-31		16	
	-22		-9		6		7		3		13		-16	
	12		28		-19		-10		-26		-10		-24	
	-3		32		-7		-28		29		7		-32	

Matrix: Secuencial
	-6		64		-14		-56		58		14		-64	
	-87		928		-203		-812		841		203		-928	
	6		-64		14		56		-58		-14		64	
	-21		224		-49		-196		203		49		-224	
	6		-64		14		56		-58		-14		64	
	-81		864		-189		-756		783		189		-864	
	-6		64		-14		-56		58		14		-64	

Matrix: Concurrente
	-6		64		-14		-56		58		14		-64	
	-87		928		-203		-812		841		203		-928	
	6		-64		14		56		-58		-14		64	
	-21		224		-49		-196		203		49		-224	
	6		-64		14		56		-58		-14		64	
	-81		864		-189		-756		783		189		-864	
	-6		64		-14		-56		58		14		-64	
Las matrices concurrente y secuencia